# Time Series specificities - Practical Application

This notebook is following the progression of the Time Series specificities class. It provides practical illustrations in Python to understand the notions we have seen in this course.

<div class="alert alert-warning">
Author: Fabrice JIMENEZ
    
Link to course materials: https://github.com/jfabrice/ml-class-time-series
</div>

## Table of contents

[Preliminary loading with Google Colab](#Preliminary-loading-with-Google-Colab)<br>
[Imports and dataset presentation](#Imports-and-dataset-presentation)<br>

[1- Formulate the question](#1--Formulate-the-question)<br>
[2- Preprocessing](#2--Preprocessing)<br>
[3- Transformation steps](#3--Transformation-steps)<br>
[4- Feature Engineering](#4--Feature-Engineering)<br>
[5- Regular approach](#5--Regular-approach)<br>
[6- Method validation](#6--Method-validation)<br>

[Conclusion](#Conclusion)

## Preliminary loading with Google Colab

If you are using this notebook with Google Colab, please execute first the following cells, to retrieve the GitHub repository content, set the working directory and install required dependencies. Otherwise, ignore these 3 cells and move to the next section.

In [ ]:
!git clone https://github.com/jfabrice/ml-class-time-series.git

In [ ]:
import os
os.chdir('ml-class-time-series')

In [ ]:
!pip install -r requirements.txt

## Imports and dataset presentation

![question](https://raw.githubusercontent.com/jfabrice/ml-class-time-series/master/data/question.PNG)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]

In [ ]:
df = pd.read_csv('data/dataset.csv')
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
print(df.shape)

In [ ]:
df.head()

How many cycles do we have? How many points per cycle?

In [ ]:
df.groupby('cycle').count()

There are 15 cycles, with very different number of points per cycle.

Let's visualize 1 parameter in time for each cycle to see how cycles are spread out.

In [ ]:
for c in df['cycle'].unique():
    df[df['cycle']==c]['p1'].plot(style='o', alpha=0.4)

Cycles seem to be separated in time by periods with no data. 

Let's see what our parameter looks like within a cycle.

In [ ]:
df[df['cycle']==1]['p1'].plot(style='o', alpha=0.4)

In [ ]:
df[df['cycle']==3]['p1'].plot(style='o', alpha=0.4)

The values seem to be continuous (close in time = close in value) which is logical if they are physical parameters.

The parameter seems to behave quite differently in different cycles.

## 1- Formulate the question

<div class="alert alert-warning">
What is the task?
</div>

"Associate cycles which are similar" : this is a <b>clustering task</b>. Each cycle will be a sample and we will group similar samples with unsupervised clustering.

"Are some cycles really different from the rest?" : this is an <b>outlier detection</b> task. We can probably use the result of the clustering to identify samples which are far away from all group centers.

<div class="alert alert-warning">
What is the scale?
</div>

Given the task, we have no choice to consider the scale of the cycle as a unit. 1 sample will be composed of 4 time series (4 parameters) within a cycle.

In addition, we saw that the number of points is very different in each cycle, so we will have to summarize each cycle into a set of features.

## 2- Preprocessing

<div class="alert alert-warning">
Are there missing values?
</div>

In [ ]:
df.isnull().sum()

<div class="alert alert-warning">
Are the time steps all equal?
</div>

In [ ]:
timescale = pd.Series(df.index)
timescale.diff().value_counts()

Time steps are not always equal to 1 second...

Let's check an example of gap of 7 seconds, to see what strategy we could use.

In [ ]:
timescale[timescale.diff() == pd.Timedelta('00:00:07')]

In [ ]:
df['p1']['2001-01-10 00:10:30':'2001-01-10 00:11:30'].plot(style='o')

We see that values still seem to be continuous. This kind of gap can be interpolated with mean value.

We assume that gaps of more than 1 day correspond to time periods between cycles. Let's confirm that.

In [ ]:
# Capture the gaps of more than 1 day
timescale[timescale.diff() >= pd.Timedelta(days=1)].reset_index()['time']

In [ ]:
# Capture the start time of each cycle
df.reset_index().groupby('cycle').min().reset_index()['time']

Indeed, the timestamps corresponding to these gaps are matching with the first point of each cycle.

<b>Conclusion</b>: We can resample data with equal steps of 1 second within each cycle, with mean interpolation (oversampling).

In [ ]:
newdf = []

# For each cycle, create a small dataframe with interpolation
for c in df['cycle'].unique():
    tmp = df[df['cycle'] == c].copy()
    tmp = tmp.resample('1S').mean().interpolate(method='linear')
    
    newdf.append(tmp)
    
# Concatenate the results to have the complete dataframe
newdf = pd.concat(newdf, axis=0)
df = newdf.copy()
del newdf

Let's check we only have time steps of 1 second or 1 day (between cycles)

In [ ]:
timescale = pd.Series(df.index)
timescale.diff().value_counts()

In [ ]:
for c in df['cycle'].unique():
    df[df['cycle']==c]['p1'].plot(style='o', alpha=0.4)

In [ ]:
df['p1']['2001-01-10 00:10:30':'2001-01-10 00:11:30'].plot(style='o')

We now have clearly defined samples (windows) which contain time series regularly indexed.

## 3- Transformation steps

<div class="alert alert-warning">
1D: Can we exploit the structure of the data for each time series ?
</div>

When we look at the parameter values closely, we have a lot of constant steps, making the overall shape piecewise constant.

In [ ]:
df[df['cycle'] == 1]['p1'][150:400].plot(style='o', alpha=0.4)

Intuitively, looking at the global shapes of signals, this kind of behavior will only add noise or pollution when we want to study the dynamics of the signal, for instance the frequencies, or the derivative. Let's filter our signals by applying a rolling mean, with a suitable window size making our signals smooth!

In [ ]:
df[df['cycle'] == 1]['p1'][150:400].rolling('10S').mean().plot(style='r-o', alpha=0.2)

Let's apply it on all cycles and all parameters.

In [ ]:
newdf = []

# For each cycle, create a small dataframe with rolling mean
for c in df['cycle'].unique():
    tmp = df[df['cycle'] == c].copy()
    tmp = tmp.rolling('10S').mean()
    
    newdf.append(tmp)
    
# Concatenate the results to have the complete dataframe
newdf = pd.concat(newdf, axis=0)
df = newdf.copy()
del newdf

As we said we want to study the dynamics of the system. Periodicity does not seem to be relevant due to the global shape of the signal across cycles. However, adding the derivative of each signal can bring information on the evolution of these signals. We keep the original signals also because the absolute quantities can also help discriminate behaviors.

In [ ]:
derivatives = []

# For each cycle, create a small dataframe with derivative values of signals
for c in df['cycle'].unique():
    tmp = df[df['cycle'] == c].diff().drop('cycle',axis=1)
    tmp.columns = ['p1_diff','p2_diff','p3_diff','p4_diff']
    
    derivatives.append(tmp)
    
# Concatenate the results to have the complete dataframe
derivatives = pd.concat(derivatives, axis=0)
df = pd.concat([df,derivatives], axis=1).dropna(axis=0)
del derivatives

In [ ]:
df.head()

<div class="alert alert-warning">
nD: Can we exploit the structure of the data for 2 or more time series at a time?
</div>

Let's look at correlations and pairplots to study pairwise relationships

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df[['p1','p2','p3','p4']].sample(300))

In [ ]:
df[['p1','p2','p3','p4']].corr()

There seems to be some correlation between parameters but not high enough to be used. Maybe PCA can give us more information by gathering all parameters. We don't need to scale our dataset because it is already scaled.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
resPCA = pca.fit_transform(df[['p1','p2','p3','p4']])
df['PC1'] = resPCA[:,0]
df['PC2'] = resPCA[:,1]

In [ ]:
# Visualize PC1 - PC2 projection for each cycle
for c in df['cycle'].unique():
    plt.plot(df[df['cycle'] == c]['PC1'], df[df['cycle'] == c]['PC2'], 'o', alpha=0.4)

Some behaviors seem to be observable on the PC1 - PC2 axes, for certain cycles but not all. It might be interesting to keep PC1 and PC2 as variables.

In the end, we have our transformed time series in our dataset with derived information.

In [ ]:
df.head()

## 4- Feature Engineering

<div class="alert alert-warning">
What features can now be used to summarize each cycle?
</div>

To describe a cycle we want to estimate:
   - The distribution of parameter values: info contained in p1, p2, p3, p4
   - The distribution of parameter combination values (pairs or more): info contained in PC1 and PC2
   - The dynamics - evolution of parameter values: info contained in p1_diff, p2_diff, p3_diff, p4_diff

We could go further, compute Fast Fourier Transform and use the coefficients, or derivatives of principal components in time, etc. But for the sake of the exercise, and considering it is a first step, we will only keep basic features on the enriched time series we already have.

In [ ]:
# Function to be applied to the dataframe of each cycle
def computeFeatures(tab):
    res = pd.Series()
    
    # For each time series we compute the following features
    for p in [c for c in tab.columns if c != 'cycle']:
        # Mean
        res[p+'_mean'] = tab[p].mean()
        # 1st decile
        res[p+'_d1'] = tab[p].quantile(0.1)
        # 9th decile
        res[p+'_d9'] = tab[p].quantile(0.9)
        # Standard deviation
        res[p+'_std'] = tab[p].std()
    
    return res

We apply the feature computation on each cycle.

In [ ]:
features = df.groupby('cycle').apply(computeFeatures)
print(features.shape)

In [ ]:
features.head()

## 5- Regular approach

<div class="alert alert-warning">
Clustering task
</div>

We have 15 cycles and 40 features. We are clearly doomed by the curse of dimensionality! Let's do a PCA to reduce the dimension, and we don't forget to scale our features, because they don't have the same units!

In [ ]:
pca = PCA(n_components=5)
resPCA = pca.fit_transform((features - features.mean())/features.std())

Let's check we have a sufficient number of components to explain most of the variance:

In [ ]:
pca.explained_variance_ratio_

Now we can perform clustering on the PCA components, let's use a hierarchical clustering to see how many groups we should have.

In [ ]:
from scipy.cluster import hierarchy

In [ ]:
hac = hierarchy.linkage(resPCA, method='ward', metric='euclidean')
dendrogram = hierarchy.dendrogram(hac, labels=features.index.map(int))

A choice of 4 clusters seems reasonable! Let's cut the dendrogram at 4 groups.

In [ ]:
features['cluster'] = hierarchy.cut_tree(hac,4).flatten()

Let's visualize the clusters in the PCA projection.

In [ ]:
colors = ['blue','red','orange','cyan']
fig, ax = plt.subplots()
for i in range(len(features)):
    ax.scatter(resPCA[i,0], resPCA[i,1], c=colors[int(features.iloc[i]['cluster'])], alpha=0.7)
    ax.text(x=resPCA[i,0]+0.1, y=resPCA[i,1]+0.1, s=str(int(features.index[i])))

Let's visualize one of the time series of each cycle in the clusters.

In [ ]:
for cluster in range(4):
    plt.figure()
    plt.title('CLUSTER '+str(cluster+1)+' - '+colors[cluster].upper()+' - PARAMETER p1')
    for c in features.index:
        if features['cluster'][c] == cluster:
            tmp = df[df['cycle'] == c]['p1'].copy()
            tmp.index = tmp.index - tmp.index.min()
            tmp.plot(style='o', alpha=0.1)
    plt.ylim([-4,5])

<div class="alert alert-warning">
Outlier detection task
</div>

Our clustering task already revealed the outliers! Cycles 3 and 8 are clear outliers. A doubt remains on cycles 1 and 4, to be studied in validation phase...

## 6- Method validation

To validate our model, we need either: 
- a validation or test set, containing the ground truth (the clusters we had to find, the real anomalies...) - <b>Strong validation strategy</b>
- a human interpretation of the results (an expert eye who is able to tell if the clusters make sense, or if the anomalies detected are real ones...) - <b>Weak validation</b>

In the frame of this class, we don't have any validation dataset, so we would rely only on a human interpretation by an expert. According to their interpretation, we can go back in the different steps of this analysis to change some elements (different resampling, smoothing, transformation steps, choice of features, method for clustering...)

This is why it is very important not to get lost in your choices! To be able to play with these choices, to adjust according to the results validation. An idea could be to keep track with a simple schema like this one:

![pipeline](https://raw.githubusercontent.com/jfabrice/ml-class-time-series/master/data/pipeline.PNG)

## Conclusion

Always think first about the principles, the intuition, the qualitative aspect behind all the available functions you can find online.

- Many people can chain very complex algorithms together and get results which might be relevant for a problem.
- Only a few can make the right choices to quickly optimize the resolution of a problem and assess its feasability...